# **Group 2 notebook**


# 1. Importing modules and defining growth coupled function

In [ ]:
import numpy, datetime, importlib, os, pickle, itertools
! pip install python-dotenv
! pip install cobra

In [ ]:
import pandas

In [ ]:
import multiprocessing, multiprocessing.pool
from multiprocessing import Process, Queue
from dotenv import find_dotenv
from os.path import dirname
import dotenv

In [ ]:
import matplotlib, matplotlib.pyplot
matplotlib.rcParams.update({'font.size':20, 'font.family':'FreeSans', 'xtick.labelsize':30, 'ytick.labelsize':30, 'axes.labelsize':40, 'figure.figsize':(12, 8)})

In [ ]:
def growth_coupled_analysis(task):
    
    """
    This function performs the growth-coupled production.
    It takes as input a list as [first_gene_pair_index, second_gene_pair_index, reaction_of_interest, biomass_reaction_label, model]
    It gives as output a list as [first_gene_pair_index, second_gene_pair_index, growth, min_production, max_production]
    """
    
    i = task[0]
    j = task[1]
    reaction_of_interest = task[2]
    biomass_reaction_label = task[3]
    model = task[4]
    
    with model as model:
                
        # KO
        model.genes[i].knock_out()
        model.genes[j].knock_out()
        solution = model.optimize()
        if solution.status == 'optimal':
            ko_growth = solution.objective_value

            # growth-coupled production
            model.objective = reaction_of_interest
            model.reactions.get_by_id(biomass_reaction_label).lower_bound = ko_growth
            max_production = model.optimize(objective_sense='maximize').objective_value
            min_production = model.optimize(objective_sense='minimize').objective_value
            
            result = [i, j, ko_growth, min_production, max_production]
        else:
            result = [i, j, 0, 0, 0]

    return result

In [ ]:
def printt(message):

    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S \t {}".format(message)))   #here date & time, to see the runtime? 

    return None

# 1. load the model


Clone model from github.

In [ ]:
! git clone https://github.com/SysBioChalmers/yeast-GEM.git

Here is the nightmare of reading the model using their methods.

In [ ]:
os.chdir('yeast-GEM')
! touch .env

# find .env + define paths:
dotenv_path = dotenv.find_dotenv()
REPO_PATH = os.path.dirname(dotenv_path)
MODEL_PATH = f"{REPO_PATH}/model/yeast-GEM.xml"

In [ ]:
!rm -rf "yeast-GEM"

In [ ]:
spec = importlib.util.spec_from_file_location("i_dont_know_what_is_this", "/content/yeast-GEM/code/io.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
model = foo.read_yeast_model()

# 2. explore the model

Define the linear solver that is used

In [ ]:
working_solver = "glpk"
model.solver = working_solver
model.solver

s_0681[e] us the id of ethanol. This was found on the github reposatory for the model 

In [ ]:
model.metabolites.get_by_id("s_0681[e]")

In [15]:
model.reactions.get_by_id('r_1761')

Reaction identifier,r_1761
Name,ethanol exchange
Memory address,0x07fc9af1c8f90
Stoichiometry,s_0681[e] --> ethanol [extracellular] -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [16]:
model.reactions.get_by_id('r_1762')

Reaction identifier,r_1762
Name,ethanol transport
Memory address,0x07fc9af1c8bd0
Stoichiometry,s_0680[c] <=> s_0681[e] ethanol [cytoplasm] <=> ethanol [extracellular]
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
model.summary()

In [19]:
number_of_genes = len(model.genes)
print(number_of_genes)

1150


Ethanol exchange is the reaction that we will couple the growth of yeast to.

In [ ]:
model.objective = 'r_1761'
model.summary()

In [ ]:
from cobra.flux_analysis import flux_variability_analysis
model.summary(fva=0.95) 

# 2. growth-coupled metabolite production exploration

## 2.1. define metabolite of interest and biomass function label

In [26]:
reaction_of_interest = 'r_1761' # ethanol export
biomass_reaction_label = 'r_2111' # biomass functions
model.objective = biomass_reaction_label
wt_solution = model.optimize()

## 2.2. run serial

- 1,225 pairs (50 x 50 genes) takes 3 min 8 sec in serial

In [ ]:
pairs = list(itertools.combinations(range(20), 2)) # generate pairs of genes
print(len(pairs)) # number of pairs generated

190


In [ ]:
%%time
results = []
for p in pairs:
  task = [p[0], p[1], reaction_of_interest, biomass_reaction_label, model]
  result = growth_coupled_analysis(task)
  results.append(result)         

CPU times: user 45.1 s, sys: 148 ms, total: 45.2 s
Wall time: 45.2 s


In [ ]:
print(len(results))
df = pandas.DataFrame(results, columns=['i', 'j', 'KO growth', 'min production', 'max production']) # i & j are the genes,number i and number j
df.head()

190


,i,j,KO growth,min production,max production
0,0,1,0.012067,1.765583,1.806006
1,0,2,0.012067,1.765583,1.806006
2,0,3,0.012067,1.765583,1.806006
3,0,4,0.012067,1.765583,1.806006
4,0,5,0.012067,1.765583,1.806006


In [ ]:
df.sort_values(by=['min production'], ascending=False).head() # 5 best in min production

,i,j,KO growth,min production,max production
64,3,14,0.012094,1.805569,1.805569
62,3,12,0.012094,1.805569,1.805569
63,3,13,0.012094,1.805569,1.805569
67,3,17,0.012094,1.805569,1.805569
59,3,9,0.012094,1.805569,1.805569


### 2.2.1. plot production envelope

In [27]:
# WT    
plotting_wt_biomass = []
wt_production = []

biomass_space = numpy.linspace(0, wt_solution.objective_value, 100)

with model as model:
    model.objective = reaction_of_interest
    for target in biomass_space:
        model.reactions.get_by_id(biomass_reaction_label).bounds = (target, target)
        solution = model.optimize()
        if solution.status == 'optimal':
            plotting_wt_biomass.append(target); wt_production.append(solution.objective_value)

In [ ]:
# KO    
#plotting optimised solution of growth-coupled production in a graph with best pair of genes for unadjusted model
#This gene pair gives the best min production of ethanol as will become clear later in the notebook
i=0; j=367

plotting_ko_biomass = []
max_productions = []
min_productions = []

with model as model:
    model.genes[i].knock_out() 
    model.genes[j].knock_out()
    ko_solution = model.optimize()
    
    biomass_space = numpy.linspace(0, ko_solution.objective_value, 100)
    with model as model:
        model.objective = reaction_of_interest
        for target in biomass_space:
            model.reactions.get_by_id(biomass_reaction_label).lower_bound = target
            max_production = model.optimize(objective_sense='maximize').objective_value
            min_production = model.optimize(objective_sense='minimize').objective_value
            plotting_ko_biomass.append(target); max_productions.append(max_production); min_productions.append(min_production)

In [ ]:
# make figure
matplotlib.pyplot.plot(plotting_wt_biomass, wt_production, '-', color='black', lw=4, label='WT')
matplotlib.pyplot.fill_between(plotting_ko_biomass, min_productions, max_productions, color='orange', alpha=0.5, label='KO')

matplotlib.pyplot.xlabel('Growth')
matplotlib.pyplot.ylabel('Production')
matplotlib.pyplot.grid(ls=':')
matplotlib.pyplot.legend()

matplotlib.pyplot.tight_layout()

## 2.3. run in parallel environment

Using multiprocessing could be difficult because if the function yields an error, it is difficult to track. Consider using testing functions and serial code as in previous section to avoid errors while executing the parallel approach.

In [ ]:
number_of_threads = 2    #here we are running all the genes in all the pairs.  before we were running 20 pairs. 

Having 1,150 genes to test implies 660,675 gene-pair evaluations.  

In in a 20 threads environment:  

- 50 x 50 genes implies 1,225 gene pairs which takes 33 sec.
- 100 x 100 genes implies 4,950 gene pairs which takes 1 min 45 sec.
- 450 x 450 genes implies 101,025 gene pairs which takes 35 min.
- all genes implies 660,675 gene pairs which takes 3h 24 min.

In [ ]:
printt('working with {} genes'.format(number_of_genes))   

tasks = []
for i in range(len(model.genes)):
    for j in range(len(model.genes)):
        if i < j:
            task = [i, j, reaction_of_interest, biomass_reaction_label, model]
            tasks.append(task)
printt('working with {} gene pairs'.format(len(tasks)))

In [ ]:
%%time
printt('entering a parallel world of {} threads'.format(number_of_threads))
hydra = multiprocessing.pool.Pool(number_of_threads)
hydra_output = hydra.map(growth_coupled_analysis, tasks)
hydra.close()
printt('completed {} tasks'.format(len(hydra_output)))

In [ ]:
df = pandas.DataFrame(hydra_output, columns=['i', 'j', 'KO growth', 'min production', 'max production'])
df.sort_values(by=['min production'], ascending=False)

# 3. store dataframe

In [ ]:
printt('store double KO information as a dataframe')

f = open('doubleKO.pickle','wb')
pickle.dump(df, f)
f.close()

# 4. adjusting the model


## 4.1 Helper functions

In [31]:
# find probes with expression log2 fold change less than or equal to -1  
import math
def find_probe_list(probe,control_list,condition_list):
  dn_probe = []
  for i in range(len(control_list)):
    FC = condition_list[i]/control_list[i]
    log_FC = math.log(FC,2)
    if(log_FC <= -1):
      dn_probe.append(probe[i])
  return dn_probe

In [32]:
# find probes with expression log2 fold change greater than or euqla to 1 
def find_probe_list_up(probe,control_list,condition_list):
  up_probe = []
  for i in range(len(control_list)):
    FC = condition_list[i]/control_list[i]
    log_FC = math.log(FC,2)
    if(log_FC >= 1):
      up_probe.append(probe[i])
  return up_probe

In [33]:
#find gene ids of probes with downregulated expression   
def find_gene_ids(dn_probe,probe_map,gene_map):
  gene_id_dn = []
  for i in range(len(dn_probe)):
    for j in range(len(probe_map)):
      if(dn_probe[i] == probe_map[j]):
        gene_id_dn.append(gene_map[j])
  return gene_id_dn

In [34]:
#make a copy of original model which is adjusted for downregulated genes
def adjust_model(model,gene_id_dn):
  reaction_ko = []
  model_adjust = model.copy()
  for gene in gene_id_dn:
    new_gene = fix_gene_id(gene)
    if new_gene in model.genes:
      model_adjust.genes.get_by_id(new_gene).knock_out()
  return model_adjust

In [35]:
#function that has as an input a gene id from the dataset and returns an id that the model can read
def fix_gene_id(gene):
  gene_new_char = []
  for char in gene:
    if char != ".":
      gene_new_char.append(char)
    else:
      break
  str = ""
  for ele in gene_new_char:
    str += ele
  return str

In [36]:
def count_ko_reactions(model_adj):
  rxn_list = []
  for i in range(len(model_adj.reactions)):
    rxn = model_adj.reactions[i]
    if rxn.bounds == (0,0):
      rxn_list.append(rxn.id)
    rxn_list
  return rxn_list

## 4.2 Loading data

Here we load and process the expression data for yeast in 3%, 7% and 0% alcohol environment

In [ ]:
! wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE20nnn/GSE20108/matrix/GSE20108_series_matrix.txt.gz
df = pandas.read_csv("GSE20108_series_matrix.txt.gz", compression="gzip", sep="\t", skiprows=59)
df.head()

In [39]:
len(df["ID_REF"])

10929

In [40]:
#make a list of all probe ids
df_probe = df.loc[:,"ID_REF"]
df_probe.head()
df_probe_list = df_probe.to_numpy()

In [42]:
#make a list of all control values by finding mean of the two replicates of the same condition
#The strains are diploid
df_control = []
df_control_list_1 = df["GSM502532"].to_numpy()
df_control_list_2 = df["GSM502533"].to_numpy()
for i in range(len(df_control_list_1)-1):
  control_i = (df_control_list_1[i]+df_control_list_2[i])/2
  df_control.append(control_i)

df_alc3 = []
df_alc3_1 = df["GSM502534"].to_numpy()
df_alc3_2 = df["GSM502535"].to_numpy()
for i in range(len(df_alc3_1)-1):
  alc3_i = (df_alc3_1[i]+df_alc3_2[i])/2
  df_alc3.append(alc3_i)

df_alc7 = []
df_alc7_1 = df["GSM502536"].to_numpy()
df_alc7_2 = df["GSM502537"].to_numpy()
for i in range(len(df_alc7_1)-1):
  alc7_i = (df_alc7_1[i]+df_alc7_2[i])/2
  df_alc7.append(alc7_i)

In [ ]:
 #File that maps probe ids to gene ids
 ! wget https://www.ebi.ac.uk/arrayexpress/files/A-AFFY-47/A-AFFY-47.adf.txt
 df2 = pandas.read_csv("A-AFFY-47.adf.txt",sep="\t", skiprows = 231)
 df2.head()

In [44]:
 len(df2["AFFX-Sc-M57289-1"])

10715

In [45]:
print(df2["AFFX-Sc-M57289-1"].nunique())

10639


In [46]:
probe_map = df2.loc[:, "RPTR-Sc-M57289-1_s_at"].to_numpy()
gene_map = df2.loc[:, "AFFX-Sc-M57289-1"].to_numpy()

## 4.3 adjust model for first and second condition

In [50]:
#adjust for 3% ethanol condition
probe_alc3 = find_probe_list(df_probe_list,df_control,df_alc3)
dn_gene_alc3 = find_gene_ids(probe_alc3, probe_map,gene_map)
model_alc3 = adjust_model(model,dn_gene_alc3)
KO_rxn_alc3 = count_ko_reactions(model_alc3)
print(len(KO_rxn_alc3))
print(KO_rxn_alc3)

19
['r_0191', 'r_0549', 'r_1075', 'r_2117', 'r_2118', 'r_2119', 'r_1119', 'r_1250', 'r_1259', 'r_1663', 'r_1795', 'r_1816', 'r_2190', 'r_2191', 'r_4062', 'r_4064', 'r_4332', 'r_4493', 'r_4660']


In [49]:
#adjust for 7% ethanol condition
probe_alc7 = find_probe_list(df_probe_list,df_control,df_alc7)
dn_gene_alc7 = find_gene_ids(probe_alc7,probe_map,gene_map)
model_alc7 = adjust_model(model,dn_gene_alc3)
KO_rxn_alc7 = count_ko_reactions(model_alc7)
print(len(KO_rxn_alc7))
print(KO_rxn_alc7)

19
['r_0191', 'r_0549', 'r_1075', 'r_2117', 'r_2118', 'r_2119', 'r_1119', 'r_1250', 'r_1259', 'r_1663', 'r_1795', 'r_1816', 'r_2190', 'r_2191', 'r_4062', 'r_4064', 'r_4332', 'r_4493', 'r_4660']


## 4.4 double KO on adjusted model

Only need to run on the 3% alcahol model since it had the same reactions knocked out as the 7% model

In [ ]:
printt('working with {} genes'.format(number_of_genes))

tasks = []
for i in range(len(model_alc3.genes)):
    for j in range(len(model_alc3.genes)):
        if i < j:
            task = [i, j, reaction_of_interest, biomass_reaction_label, model_alc3]
            tasks.append(task)
printt('working with {} gene pairs'.format(len(tasks)))

In [ ]:
%%time
printt('entering a parallel world of {} threads'.format(number_of_threads))
hydra = multiprocessing.pool.Pool(number_of_threads)
hydra_output = hydra.map(growth_coupled_analysis, tasks)
hydra.close()
printt('completed {} tasks'.format(len(hydra_output)))

In [ ]:
df = pandas.DataFrame(hydra_output, columns=['i', 'j', 'KO growth', 'min production', 'max production'])
df.sort_values(by=['min production'], ascending=False)

In [ ]:
printt('store double KO information as a dataframe')

f = open('doubleKO_alc3.pickle','wb')
pickle.dump(df, f)
f.close()

# 5. Exploring genes that give growth coupled design

## 5.1 unadjusted model

In [54]:
#upload unadjusted json file here, it contains results from running growth coupled function on the unadjusted model
import json
with open('/content/doubleKO_unadjusted.json', 'r') as f:
    new_str = json.load(f)
df_un = pandas.read_json(new_str)

In [55]:
df_un.head()
df_un.sort_values("min production",ascending = False)

,i,j,KO growth,min production,max production
366,0,367,0.011881,1.817702,1.817702
7685,6,813,0.011881,1.817701,1.817701
812,0,813,0.011881,1.817701,1.817701
142685,131,813,0.011881,1.817701,1.817701
7239,6,367,0.011881,1.817700,1.817701
...,...,...,...,...,...
224163,215,349,0.083748,0.000000,0.000000
224164,215,350,0.083748,0.000000,0.000000
224165,215,351,0.083748,0.000000,0.000000
224166,215,352,0.083743,0.000000,0.000000


In [56]:
df_un.loc[df_un["min production"]>0]

,i,j,KO growth,min production,max production
0,0,1,0.012067,1.765583,1.806007
1,0,2,0.012067,1.765583,1.806007
2,0,3,0.012067,1.765478,1.806030
3,0,4,0.012067,1.765583,1.806007
4,0,5,0.012067,1.765478,1.806030
...,...,...,...,...,...
631509,907,1145,0.012094,1.805569,1.805569
631510,907,1146,0.012095,1.805571,1.805571
631511,907,1147,0.012095,1.805571,1.805571
631512,907,1148,0.012095,1.805570,1.805570


In [57]:
df_gene_un = df_un.loc[df_un["min production"] >= 1.8]
df_gene_un.sort_values("min production",ascending = False)

,i,j,KO growth,min production,max production
366,0,367,0.011881,1.817702,1.817702
7685,6,813,0.011881,1.817701,1.817701
812,0,813,0.011881,1.817701,1.817701
142685,131,813,0.011881,1.817701,1.817701
7239,6,367,0.011881,1.817700,1.817701
...,...,...,...,...,...
299498,299,798,0.012050,1.801073,1.801073
29197,25,798,0.012050,1.801073,1.801073
373420,391,798,0.012050,1.801073,1.801073
436208,479,798,0.012050,1.801073,1.801073


In [64]:
i_gene_un = df_gene_un["i"].tolist()
j_gene_un = df_gene_un["j"].tolist()
gene_list_un = j_gene_un + i_gene_un

In [59]:
from collections import Counter
c_un = Counter(gene_list_un)
print(c_un.most_common(5))

[(3, 964), (299, 958), (238, 956), (789, 956), (231, 955)]


In [ ]:
print(model.genes[3])
print(model.genes[299])
print(model.genes[238])
print(model.genes[789])
print(model.genes[231])

In [61]:
print("4 best gene pairs:")
for gene_num in range(4):
  print(str(model.genes[i_gene_un[gene_num]]) + " and " + str(model.genes[j_gene_un[gene_num]]))

4 best gene pairs:
Q0045 and YEL047C
Q0045 and YGR208W
Q0045 and YLR011W
Q0045 and YOL126C


In [65]:
print(model.genes[0])
print(model.genes[367])

Q0045
YGR208W


## 5.2 adjusted model with downregulated genes KO

In [66]:
import json
with open('/content/doubleKO_adjusted_dn.json', 'r') as f:
    new_str = json.load(f)
df_dn = pandas.read_json(new_str)
df_dn.sort_values("min production", ascending = False)

,i,j,KO growth,min production,max production
245710,238,690,0,0.1,1.06
245548,238,528,0,0.1,1.06
142562,131,690,0,0.1,1.06
7038,6,166,0,0.1,1.06
6419,5,690,0,0.1,1.06
...,...,...,...,...,...
660672,1147,1148,0,0.0,2.00
660673,1147,1149,0,0.0,2.00
660657,1143,1147,0,0.0,2.00
660655,1143,1145,0,0.0,2.00


In [67]:
df_gene_dn = df_dn.loc[df_dn["min production"] >= 0.1]
df_gene_dn.sort_values("min production",ascending = False)

,i,j,KO growth,min production,max production
245710,238,690,0,0.1,1.06
142562,131,690,0,0.1,1.06
6419,5,690,0,0.1,1.06
7562,6,690,0,0.1,1.06
4130,3,690,0,0.1,1.06
29089,25,690,0,0.1,1.06
239312,231,690,0,0.1,1.06
348560,359,690,0,0.1,1.06
467804,528,789,0,0.1,1.06
299228,299,528,0,0.1,1.06


In [68]:
len(df_dn.loc[df_dn["min production"] > 0])


32

In [69]:
len(df_gene_dn)

32

In [70]:
i_gene_dn = df_gene_dn["i"].tolist()
j_gene_dn = df_gene_dn["j"].tolist()
gene_list_dn = j_gene_dn + i_gene_dn

In [71]:
from collections import Counter
c_dn = Counter(gene_list_dn)
print(c_dn.most_common(5))

[(690, 14), (528, 14), (166, 4), (131, 3), (0, 3)]


In [72]:
print(model.genes[690])
print(model.genes[528])
print(model.genes[166])
print(model.genes[131])
print(model.genes[0])

YMR205C
YKL060C
YDR050C
YDL067C
Q0045


In [ ]:
print("4 best gene pairs:")
for gene_num in range(4):
  print(str(model.genes[i_gene_dn[gene_num]]) + " and " + str(model.genes[j_gene_dn[gene_num]]))

plotting

In [74]:
wt_solution = model_alc3.optimize()
print(wt_solution.objective_value)

-2.7200032349665246e-16


In [75]:
number_of_genes = len(model_alc3.genes)
print(number_of_genes)

1150


## 5.3 adjusted model with upregulated genes

In [ ]:
import json
with open('/content/doubleKO_adjusted_up.json', 'r') as f:
    new_str = json.load(f)
df_up = pandas.read_json(new_str)
df_up.sort_values("min production", ascending = False)

In [87]:
df_gene_up = df_up.loc[df_up["min production"] >= 1.8]
df_gene_up.sort_values("min production",ascending = False)

,i,j,KO growth,min production,max production
6096,5,367,0.011881,1.817701,1.817701
7239,6,367,0.011881,1.817701,1.817701
6542,5,813,0.011881,1.817701,1.817701
7685,6,813,0.011881,1.817701,1.817701
366,0,367,0.011881,1.817700,1.817700
...,...,...,...,...,...
299498,299,798,0.012050,1.801073,1.801073
599007,798,907,0.012050,1.801073,1.801073
4238,3,798,0.012050,1.801073,1.801073
373420,391,798,0.012050,1.801073,1.801073


In [86]:
len(df_gene_up.loc[df_gene_up["min production"] > 0])

9527

In [82]:
i_gene_up = df_gene_up["i"].tolist()
j_gene_up = df_gene_up["j"].tolist()
gene_list_up = j_gene_up + i_gene_up

In [83]:
from collections import Counter
c_up = Counter(gene_list_up)
print(c_up.most_common(5))
print(c_un.most_common(5))

[(25, 961), (299, 958), (238, 957), (359, 957), (3, 957)]
[(3, 964), (299, 958), (238, 956), (789, 956), (231, 955)]


In [84]:
print(model.genes[25])
print(model.genes[299])
print(model.genes[238])
print(model.genes[359])
print(model.genes[3])

YBL045C
YFR033C
YEL024W
YGR183C
Q0105


In [85]:
print("4 best gene pairs:")
for gene_num in range(4):
  print(str(model.genes[i_gene_up[gene_num]]) + " and " + str(model.genes[j_gene_up[gene_num]]))

4 best gene pairs:
Q0250 and YGR208W
Q0275 and YGR208W
Q0250 and YOR184W
Q0275 and YOR184W


In [93]:
probe_alc3 = find_probe_list_up(df_probe_list,df_control,df_alc3)
up_gene_alc3 = find_gene_ids(probe_alc3, probe_map,gene_map)
model_alc3_up = adjust_model(model,up_gene_alc3)
KO_rxn_up = count_ko_reactions(model_alc3_up)
print(KO_rxn_up)

['r_0004', 'r_0694', 'r_0695', 'r_0763', 'r_0940', 'r_1045', 'r_2194', 'r_2200', 'r_3304', 'r_3305', 'r_3306', 'r_3307', 'r_1102', 'r_1169', 'r_1170', 'r_1236', 'r_1250', 'r_1259', 'r_2219', 'r_2220', 'r_2221', 'r_2222', 'r_2223', 'r_2224', 'r_2225', 'r_2226', 'r_2227', 'r_2228', 'r_1663', 'r_1771', 'r_1772', 'r_1774', 'r_2231', 'r_3606', 'r_3607', 'r_4062', 'r_4064', 'r_4279']


In [94]:
wt_solution = model_alc3_up.optimize()
print(wt_solution.objective_value)

0.0
